# RF Chain and Linearity Commissioning Test
copied from system health displays NB to be updated

## Table of Contents
#### [1 - Global Configuration](#1-global-configuration)
#### [2 - Setup Test Equipment](#2-test-equipment-setup)
#### [3 - Configure Scan Tests](#3-configure-scan-tests)
#### [4 - Debug Telescope](#4-debug-telescope)
#### [5 - Dashboard Links](#5-dashboard-links)
#### [6 - System Health Displays Tests](#6-system-health-displays-tests)

## 1 Global Configuration

### 1.1 Import dependencies

In [1]:
import sys

sys.path.append("../../src")

import json
import logging
import os
import pathlib
import time
import tango

import ska_ser_logging
from bokeh.io import output_notebook
from ska_oso_pdm.entities.common.target import (
    CrossScanParameters,
    FivePointParameters,
    RasterParameters,
    SinglePointParameters,
    StarRasterParameters,
)
from ska_oso_pdm.entities.sdp import BeamMapping
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions.devicecontrol.resource_control import get_request_json
from ska_oso_scripting.objects import SubArray, Telescope
from ska_tmc_cdm.messages.central_node.assign_resources import AssignResourcesRequest
from ska_tmc_cdm.messages.central_node.sdp import Channel
from ska_tmc_cdm.messages.subarray_node.configure import ConfigureRequest
from ska_tmc_cdm.messages.subarray_node.configure.core import ReceiverBand

from ska_mid_jupyter_notebooks.cluster.cluster import Environment, TangoDeployment
from ska_mid_jupyter_notebooks.dish.dish import TangoDishDeployment
from ska_mid_jupyter_notebooks.helpers.path import project_root
from ska_mid_jupyter_notebooks.obsconfig.config import ObservationSB
from ska_mid_jupyter_notebooks.obsconfig.target_spec import TargetSpec
from ska_mid_jupyter_notebooks.sut.rendering import TelescopeMononitorPlot
from ska_mid_jupyter_notebooks.sut.state import TelescopeDeviceModel, get_telescope_state
from ska_mid_jupyter_notebooks.sut.sut import TangoSUTDeployment, disable_qa
from ska_mid_jupyter_notebooks.test_equipment.rendering import get_test_equipment_monitor_plot
from ska_mid_jupyter_notebooks.test_equipment.state import get_equipment_model
from ska_mid_jupyter_notebooks.test_equipment.test_equipment import TangoTestEquipment
print("Done")

Done


### 1.2 Setup Global Variables and Configuration

In [2]:
# print(f"DISH_LMC_IN_THE_LOOP={os.environ('DISH_LMC_IN_THE_LOOP')}")
debug_mode = True  # This setting enables printing of diagnostics
enable_logging = False  # This enables logging and sets the global log_level to debug
dishlmc_enabled = True  # Set this to true if you have a dish LMC deployment
executon_environment = Environment.Integration
branch_name = None  # Set this if you are using an on-demand deployment (i.e. Environment.CI)
if enable_logging:
    ska_ser_logging.configure_logging(logging.DEBUG)
else:
    ska_ser_logging.configure_logging(logging.INFO)
test_equipment = TangoTestEquipment()
print(f"Test Equipment Configured: {test_equipment}\n")
# namespace_override parameter can be used to override auto-configured SUT namespace
sut_namespace_override = ""
subarray_count = 1
subarray_id = 1
sut = TangoSUTDeployment(
    branch_name,
    executon_environment,
    namespace_override=sut_namespace_override,
    subarray_index=subarray_id,
)
print(f"SUT configured: {str(sut)}\n")
dish_ids = ["0001", "0036"]
# namespace_override parameter can be used to override auto-configured dish namespace
dish_namespace_overrides = ["", ""]
if dishlmc_enabled:
    dish_deployments = []
    for i, d in enumerate(dish_ids):
        dish = TangoDishDeployment(
            f"ska{d[1:]}",
            branch_name=branch_name,
            environment=executon_environment,
            namespace_override=dish_namespace_overrides[i],
        )
        print(f"Dish {d} configured: {dish}\n")
        dish_deployments.append(dish)
else:
    dish_deployments = []

timestr = time.strftime("%Y%m%d-%H%M")
notebook_output_dir = pathlib.Path(
    project_root(), f"notebook-execution-data/configure_scan_for_commissioning/execution-{timestr}"
)
os.makedirs(notebook_output_dir)
# we disable qa as it is not been properly verified
disable_qa()
print("Done")

Test Equipment Configured: TangoTestEquipment{Deployment: namespace=test-equipment; tango_host=tango-databaseds.test-equipment.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.test-equipment.svc.miditf.internal.skao.int:8765}

SUT configured: TangoSUTDeployment{subarray_index=1; Deployment: namespace=integration; tango_host=tango-databaseds.integration.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration.svc.miditf.internal.skao.int:8765}

Dish 0001 configured: TangoDishDeployment{dish_id=ska001; Deployment: namespace=integration-dish-lmc-ska001; tango_host=tango-databaseds.integration-dish-lmc-ska001.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration-dish-lmc-ska001.svc.miditf.internal.skao.int:8765}

Dish 0036 configured: TangoDishDeployment{dish_id=ska036; Deployment: namespace=integratio

### 1.3 Test Connections to Namespaces

In [3]:
sut.smoke_test()
test_equipment.smoke_test()
for dish_deployment in dish_deployments:
    dish_deployment.smoke_test()
print("Done")

CIA PingResponse (integration): {"result":"ok","time":"2024-05-13T11:46:30.687451"}
CIA PingResponse (test-equipment): {"result":"ok","time":"2024-05-13T11:46:31.352086"}
mid-itf/siggen/1 is reachable
mid-itf/progattenuator/1 is reachable
mid-itf/spectana/1 is reachable
mid-itf/skysimctl/4 is reachable
CIA PingResponse (integration-dish-lmc-ska001): {"result":"ok","time":"2024-05-13T11:46:33.026220"}
CIA PingResponse (integration-dish-lmc-ska036): {"result":"ok","time":"2024-05-13T11:46:33.692310"}
Done


### 1.4 Export System Configuration

In [4]:
deployment: TangoDeployment
for deployment in [sut, test_equipment, *dish_deployments]:
    deployment.export_chart_configuration(output_dir=notebook_output_dir)
print("Done")

Exporting configuration using http://config-inspector.integration.svc.miditf.internal.skao.int:8765
ReleaseResponse (integration): {
    "chart": "ska-mid-itf",
    "version": "22.0.1",
    "sub_charts": [
        {
            "chart": "ska-tango-base",
            "version": "0.4.9"
        },
        {
            "chart": "ska-tango-util",
            "version": "0.4.10"
        },
        {
            "chart": "ska-tmc-mid",
            "version": "0.19.1"
        },
        {
            "chart": "ska-csp-lmc-mid",
            "version": "0.18.2"
        },
        {
            "chart": "ska-mid-cbf-mcs",
            "version": "0.14.1"
        },
        {
            "chart": "ska-sdp",
            "version": "0.19.1"
        },
        {
            "chart": "ska-ser-config-inspector",
            "version": "0.2.2"
        },
        {
            "chart": "ska-tango-taranta",
            "version": "2.8.3"
        },
        {
            "chart": "ska-tango-tangogql",
   

## 2 Test Equipment Setup

Use the noise source at nominal levels for input to at least one SPFRx.  

### 2.1 Configure Test Equipment State 

In [5]:
test_equipment_state = get_equipment_model(test_equipment)
test_equipment.devices
print("Done")

Done


### 2.2 Print Test Equipment Diagnostics

In [6]:
test_equipment.print_diagnostics()
print("Done")

mid-itf/skysimctl/4 State: ON
mid-itf/skysimctl/4 Band: 1
mid-itf/skysimctl/4 Correlated_Noise_Source: True
mid-itf/skysimctl/4 Uncorrelated_Noise_Sources: False
mid-itf/skysimctl/4 H_Channel: False
mid-itf/skysimctl/4 V_Channel: False
mid-itf/skysimctl/4 temperature: 24.0
mid-itf/skysimctl/4 humidity: 0.0
mid-itf/siggen/1 versionId: 0.9.1
mid-itf/siggen/1 adminMode: 0
mid-itf/siggen/1 State: ON
mid-itf/siggen/1 healthState: HealthState.UNKNOWN
mid-itf/siggen/1 frequency: 800000000.0
mid-itf/siggen/1 power_cycled: False
mid-itf/siggen/1 power_dbm: -30.0
mid-itf/siggen/1 rf_output_on: True
mid-itf/siggen/1 controlMode: 0
mid-itf/siggen/1 simulationMode: 0
mid-itf/siggen/1 testMode: 0
mid-itf/siggen/1 loggingLevel: 4
mid-itf/siggen/1 command_error: False
mid-itf/siggen/1 device_error: False
mid-itf/siggen/1 execution_error: False
mid-itf/siggen/1 query_error: False
mid-itf/spectana/1 adminMode: 0
mid-itf/spectana/1 State: ON
mid-itf/spectana/1 attenuation: 15
mid-itf/spectana/1 frequency

mid-itf/progattenuator/1 channel_1: 10.0
mid-itf/progattenuator/1 controlMode: 0
mid-itf/progattenuator/1 healthState: HealthState.UNKNOWN
mid-itf/progattenuator/1 loggingLevel: 5
mid-itf/progattenuator/1 simulationMode: 0
mid-itf/progattenuator/1 testMode: 0
Done


### 2.3 Create Test Equipment Plot

In [8]:
monitor_plot = get_test_equipment_monitor_plot()
test_equipment_state.subscribe_to_test_equipment_state(monitor_plot.handle_device_state_change)
output_notebook()
monitor_plot.show()
test_equipment_state.activate()
print("Done")

Loading BokehJS ...

Done


### 2.4 Turn offline Test Equipment devices ONLINE

In [9]:
# set any offline devices to online
test_equipment.turn_online()
print("Done")

set mid-itf/siggen/1 adminMode already ONLINE
set mid-itf/progattenuator/1 adminMode already ONLINE
set mid-itf/spectana/1 adminMode already ONLINE
Done


### 2.5 Display Test Equipment Device States 

In [10]:
test_equipment_state.state["devices_states"]
print("Done")

Done


### 2.6 Configure Signal Generator and set noise

In [11]:
frequency_to_set = 880e6

signal_generator = test_equipment.signal_generator
print(f"Current signal generator frequency: {signal_generator.frequency}")
signal_generator.write_attribute("frequency", frequency_to_set)
time.sleep(1)
print(f"Updated signal generator frequency: {signal_generator.frequency}")
assert (
    signal_generator.frequency == frequency_to_set
), f"Frequency required is {frequency_to_set} but got {signal_generator.frequency}"
print("Done")

Current signal generator frequency: 800000000.0
Updated signal generator frequency: 880000000.0
Done


## 3 Configure Scan Tests

Use OSO scripting interface to TMC to run the script for assigning resources for a single sub-array, configuring and running a  scan.  

### 3.1 Setup Telescope Monitoring

#### 3.1.1 Configure Telescope Monitoring

In [12]:
# setup monitoring
# use telescope state object for state monitoring
print(f"Monitor {subarray_count} subarrays with dish IDs {dish_ids}")
device_model = TelescopeDeviceModel(dish_ids, subarray_count)
try:
    print(f"Get telescope state for device model {device_model}: sysytem under test {sut}")
    telescope_state = get_telescope_state(device_model, sut)
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
    telescope_state = None
telescope_monitor_plot = TelescopeMononitorPlot(plot_width=900, plot_height=200)
if telescope_state is not None:
    print(f"Telecope state {telescope_state}")
    # use monitor plot as a dashboard
    # set up events to monitor
    telescope_state.subscribe_to_on_off(telescope_monitor_plot.observe_telescope_on_off)
    telescope_state.subscribe_to_subarray_resource_state(
        telescope_monitor_plot.observe_subarray_resources_state
    )
    telescope_state.subscribe_to_subarray_configurational_state(
        telescope_monitor_plot.observe_subarray_configuration_state
    )
    telescope_state.subscribe_to_subarray_scanning_state(
        telescope_monitor_plot.observe_subarray_scanning_state
    ) 
else:
    print("ERROR: telescope state is unknown")
output_notebook()
print("Done")

Monitor 1 subarrays with dish IDs ['0001', '0036']
Get telescope state for device model Telecope Dish IDs ['0001', '0036']: sysytem under test TangoSUTDeployment{subarray_index=1; Deployment: namespace=integration; tango_host=tango-databaseds.integration.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration.svc.miditf.internal.skao.int:8765}
Telecope state Telescope:
Telecope Dish IDs ['0001', '0036']
TangoSUTDeployment{subarray_index=1; Deployment: namespace=integration; tango_host=tango-databaseds.integration.svc.miditf.internal.skao.int:10000; cluster_domain=miditf.internal.skao.int; cia_url=http://config-inspector.integration.svc.miditf.internal.skao.int:8765}


Loading BokehJS ...

Done


#### 3.1.2 Open the inline dashboard
Start the simple inline dashboard showing current state of the Telescope and resource assignment and configuration status.

In [13]:
telescope_monitor_plot.show()
telescope_state.activate()
telescope_state.wait_til_ready(2)
print("Done")

OK


### 3.2 Print System Diagnostics

#### 3.2.1 Print TMC Diagnostics

In [14]:
sut.print_tmc_diagnostics()
print("Done")

TMC Central Node state: ON
TMC Central Node adminMode: AdminMode.OFFLINE
TMC Central Node healthState: HealthState.OK
TMC Central Node telescopeHealthState: HealthState.UNKNOWN
TMC Central Node isDishVccConfig: True
TMC Central Node dishvccvalidationstatus: {"dish": "ALL DISH OK", "ska_mid/tm_leaf_node/csp_master": "TMC and CSP Master Dish Vcc Version is Same"}
TMC Subarray Node state: ON
TMC Subarray adminMode: AdminMode.OFFLINE
TMC Subarray Node obsState: ObsState.SCANNING
Done


#### 3.2.2 Print CSP-LMC Diagnostics

In [15]:
sut.print_csp_diagnostics()
print("Done")

CSP-LMC Controller adminMode: AdminMode.ONLINE
CSP-LMC Controller State: ON
CSP-LMC Controller dishVccConfig: {"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 119}, "SKA036": {"vcc": 2, "k": 1127}, "SKA063": {"vcc": 3, "k": 620}, "SKA100": {"vcc": 4, "k": 101}}}
CSP-LMC Controller CBFSimulationMode: True
CSP-LMC Subarray adminMode: AdminMode.ONLINE
CSP-LMC Subarray State: ON
CSP-LMC Subarray obsState: ObsState.READY
CSP-LMC Subarray dishVccConfig: {"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 119}, "SKA036": {"vcc": 2, "k": 1127}, "SKA063": {"vcc": 3, "k": 620}, "SKA100": {"vcc": 4, "k": 101}}}
Done


#### 3.2.3 Print CBF Diagnostics

In [16]:
sut.print_cbf_diagnostics()
print("Done")

CBF Controller adminMode: AdminMode.ONLINE
CBF Controller State: ON
CBF Subarray adminMode: AdminMode.ONLINE
CBF Subarray State: ON
CBF Subarray obsState: ObsState.READY
Done


#### 3.2.4 Print SDP Diagnostics

In [17]:
sut.print_sdp_diagnostics()
print("Done")

SDP Controller state: ON
SDP Controller adminMode: AdminMode.ONLINE
SDP Subarray state: ON
SDP Subarray adminMode: AdminMode.ONLINE
SDP Subarray obsState: ObsState.READY
Done


#### 3.2.5 Print Dish-LMC Diagnostics

In [18]:
for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id} - {dish_deployment.namespace}: Diagnostics")

    dish_deployment.print_diagnostics()
print("Done")

Dish ska001 - integration-dish-lmc-ska001: Diagnostics
ska001: ComponentStates: "{'SPF': {'operatingmode': <SPFOperatingMode.OPERATE: 3>, 'powerstate': <SPFPowerState.FULL_POWER: 2>, 'healthstate': <HealthState.UNKNOWN: 3>, 'bandinfocus': <SPFBandInFocus.B2: 2>, 'b1capabilitystate': <SPFCapabilityStates.OPERATE_FULL: 3>, 'b2capabilitystate': <SPFCapabilityStates.OPERATE_FULL: 3>, 'b3capabilitystate': <SPFCapabilityStates.OPERATE_FULL: 3>, 'b4capabilitystate': <SPFCapabilityStates.OPERATE_FULL: 3>, 'b5acapabilitystate': <SPFCapabilityStates.OPERATE_FULL: 3>, 'b5bcapabilitystate': <SPFCapabilityStates.OPERATE_FULL: 3>}, 'DS': {'healthstate': <HealthState.UNKNOWN: 3>, 'operatingmode': <DSOperatingMode.POINT: 7>, 'pointingstate': <PointingState.TRACK: 2>, 'achievedtargetlock': None, 'indexerposition': <IndexerPosition.B2: 2>, 'powerstate': <DSPowerState.FULL_POWER: 2>, 'desiredpointingaz': [1715580700.0, 0.0], 'desiredpointingel': [1715580700.0, 0.0], 'achievedpointing': [1715601300.0, 255

#### 3.2.6 Print Full System Diagnostics

In [ ]:
print("SUT: Full Diagnostics")
sut.print_full_diagnostics()

for dish_deployment in dish_deployments:
    print(f"Dish {dish_deployment.dish_id}: Full Diagnostics")
    dish_deployment.print_full_diagnostics()

print("Test Equipment: Full Diagnostics")
test_equipment.print_full_diagnostics()
print("OK")

### 3.3 Setup ODA

In [ ]:
os.environ["ODA_URI"] = (
    "http://ingress-nginx-controller-lb-default.ingress-nginx.svc.miditf.internal.skao.int/ska-db-oda/api/v1/"
)
eb_id = oda_helper.create_eb()
print(f"Execution Block ID: {eb_id}")
print("OK")

### 3.4 Initialise Telescope and Subarray
Create Subarray and Telescope instances.

In [ ]:
sub = SubArray(subarray_id)
tel = Telescope()
print("OK")

### 3.5 Dish-VCC Configuration in TMC

#### 3.5.1 Check Dish-VCC Configuration in TMC
Loading fails because the telescope is not in the correct state.

In [26]:
tmc_obs = sut.tmc_subarray.obs_state
csp_obs = sut.csp_subarray.obs_state
sdp_obs = sut.sdp_subarray.obs_state
print(f"TMC subarray state {str(tmc_obs)} ({tmc_obs})")
print(f"CSP subarray state {str(csp_obs)} ({csp_obs})")
print(f"SDP subarray state {str(sdp_obs)} ({sdp_obs})")
if tmc_obs != 4:
    print("ERROR: TMC subarray is not ready")
if csp_obs != 4:
    print("ERROR: CSP subarray is not ready")
if sdp_obs != 4:
    print("ERROR: SDP subarray is not ready")

TMC subarray state ObsState.SCANNING (5)
CSP subarray state ObsState.READY (4)
SDP subarray state ObsState.READY (4)
ERROR: TMC subarray is not ready


#### 3.5.2 Load Dish-VCC Configuration in TMC

In [30]:
# This should only be executed for a fresh deployment (i.e. Telescope is OFF.
# If you have restarted the subarray, you should not run this command
try:
    sut.load_dish_vcc_config()
    print("OK")
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
except KeyboardInterrupt:
    print("Terminated")
except TimeoutError as tmer:
    printprint(f"ERROR: {tmer}")
print("Done")

CSP Controller: adminMode=0; State=ON
CSP Controller: adminMode=0; State=ON after 0s.
CSP Controller: adminMode=0; State=ON after 1s.
CSP Controller: adminMode=0; State=ON after 3s.
CSP Controller: adminMode=0; State=ON after 7s.
CSP Controller: adminMode=0; State=ON after 15s.
CSP Controller: adminMode=0; State=ON after 31s.
CSP Controller: adminMode=0; State=ON after 63s.
1|2024-05-13T12:36:46.948Z|WARNING|Thread-7 (_listening_daemon)|_listening_daemon|statemonitoring.py#1138||exiting monitoring loop due to an unknown exception


Exception in thread Thread-7 (_listening_daemon):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/skao/ska_mid_jupyter_notebooks/monitoring/statemonitoring.py", line 1139, in _listening_daemon
    raise exception
  File "/usr/local/lib/skao/ska_mid_jupyter_notebooks/monitoring/statemonitoring.py", line 1110, in _listening_daemon
    event_key_input = event.key
  File "/usr/local/lib/skao/ska_mid_jupyter_notebooks/monitoring/statemonitoring.py", line 75, in key
    return event_key(self.device.name(), self.attr_name)
PyTango.ConnectionFailed: DevFailed[
DevError[
    desc = TRANSIENT CORBA system exception: TRANSIENT_ConnectFailed
  origin = void Tango::Connection::connect(const string&) at (/src/cppTango/cppapi/client/devapi_base.cpp:604)
  reason = API_CorbaException
severity = ERR]


ERROR: Failed to connect to device sys/database/2
The connection request was delayed.
The last connection request was done less than 1000 ms ago
Done


### 3.6 Turn telescope ON

In [ ]:
# set to ON only if OFF
# If you have restarted the subarray, you should not run this command (Telescope is already ON)
# dish_lmc mode must be in LP_standby and before trying to turn the telescope ON
# Takes about 1m20s
if telescope_monitor_plot.on_off_state == "OFF":  # e.g. purple
    tel.on()
else:
    assert (
        telescope_monitor_plot.on_off_state == "ON"
    ), f"Cant continue with telescope in {telescope_monitor_plot.on_off_state}"
print("OK")

### 3.7 Observation Definition

#### 3.7.1 Define Resources to be used during Observation
Generate Processing Block and Execution Block IDs using SKUID.

In [ ]:
# External IDs (this would typically come from a database like ODA)
# this is simulated by means of instantiate a new Observation object comping from the helper modules
observation = ObservationSB()
print("OK")

#### 3.7.2 Create the high level observation specifications in terms of target specs

Note :- Users may currently modify the values by replacing the example values as given for each field within Target specification section.

In [ ]:
observation = ObservationSB()
dish_ids = [d.dish_id.upper() for d in dish_deployments]
DEFAULT_TARGET_SPECS = {
    "flux calibrator": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="flux calibrator",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels9",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
    "M87": TargetSpec(
        target_sb_detail={
            "co_ordinate_type": "Equatorial",
            "ra": "19:24:51.05 degrees",
            "dec": "-29:14:30.12 degrees",
            "reference_frame": "ICRS",
            "unit": ("hourangle", "deg"),
            "pointing_pattern_type": {
                "single_pointing_parameters": SinglePointParameters(
                    offset_x_arcsec=0.0, offset_y_arcsec=0.0
                ),
                "raster_parameters": RasterParameters(
                    row_length_arcsec=0.0,
                    row_offset_arcsec=0.0,
                    n_rows=1,
                    pa=0.0,
                    unidirectional=False,
                ),
                "star_raster_parameters": StarRasterParameters(
                    row_length_arcsec=0.0,
                    n_rows=1,
                    row_offset_angle=0.0,
                    unidirectional=False,
                ),
                "five_point_parameters": FivePointParameters(offset_arcsec=0.0),
                "cross_scan_parameters": CrossScanParameters(offset_arcsec=0.0),
                "active_pointing_pattern_type": "single_pointing_parameters",
            },
        },
        scan_type="M87",
        band=ReceiverBand.BAND_2,
        channelisation="vis_channels10",
        polarisation="all",
        processing="test-receive-addresses",
        dish_ids=dish_ids,
        target=None,
    ),
}


channel_configuration = [
    Channel(
        spectral_window_id="fsp_1_channels",
        count=14880,
        start=0,
        stride=2,
        freq_min=0.35e9,
        freq_max=0.368e9,
        link_map=[[0, 0], [200, 1], [744, 2], [944, 3]],
    )
]

for key, value in DEFAULT_TARGET_SPECS.items():
    observation.add_channel_configuration(value.channelisation, channel_configuration)

observation.add_target_specs(DEFAULT_TARGET_SPECS)

for target_id, target in DEFAULT_TARGET_SPECS.items():
    observation.add_scan_type_configuration(
        config_name=target_id,
        beams={"vis0": BeamMapping(beam_id="vis0", field_id="M83")},
        derive_from=".default",
    )
scan_def_id = "flux calibrator"
observation.add_scan_sequence([scan_def_id])
print("OK")

#### 3.7.3 Mid configuration schema input used by observing commands

[Configuration Schemas-OET→TMC(Mid)](https://developer.skao.int/projects/ska-telmodel/en/latest/)

In [ ]:
telescope_monitor_plot.show()
print("OK")

#### 3.7.4 Create Scheduling Block Definition(SBD) Instance and save it into the ODA

In [ ]:
observation.eb_id = eb_id
pdm_allocation = observation.generate_pdm_object_for_sbd_save(DEFAULT_TARGET_SPECS)

sbd = oda_helper.save(pdm_allocation)
sbd_id = sbd.sbd_id
pdm_allocation.sbd_id = sbd_id
print(f"Saved Scheduling Block Definition Instance in ODA: SBD_ID={sbd_id}")
print("OK")

### 3.8 Assign Resources
Assign the requested resources to a Subarray

In [ ]:
print("Assign request")
assign_request = observation.generate_allocate_config_sb(pdm_allocation).as_object
print(f"Got assign request {assign_request}")
print()

if debug_mode:
    print("Debug request JSON")
    request_json = get_request_json(assign_request, AssignResourcesRequest, True)
    print("AssignResourcesRequest:", json.dumps(json.loads(request_json), indent=2))
print()

print("Assign from Control Data Model (CDM) from subarray {sub.id}")
try:
    sub.assign_from_cdm(assign_request, timeout=120)
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
except EventTimeoutError:
    print("Could not assign resources")
print()

print("Plot telescope monitor")
try:
    telescope_monitor_plot.show()
except tango.DevFailed as terr:
    print(f"ERROR: {terr.args[0].desc.strip()}")
print()

print("OK")

In [ ]:
telescope_monitor_plot.show()
print("OK")

### 3.9 Configure Scan
Configure the telescope  on first target in sequence - may be modified to configure and run multiple targets at a later time.

In [ ]:
configure_object = observation.generate_scan_config_sb(
    pdm_observation_request=pdm_allocation,
    scan_definition_id=scan_def_id,
    scan_duration=10.0,
).as_object

if debug_mode:
    cfg_json = get_request_json(configure_object, ConfigureRequest)
    print(f"ConfigureRequest={cfg_json}")

sub.configure_from_cdm(configure_object, timeout=120)
time.sleep(2)
print("OK")

In [ ]:
telescope_monitor_plot.show()
print("OK")

### 3.10 Run the Scan

In [ ]:
sub.scan(timeout=120)
telescope_monitor_plot.show()
print("OK")

### 3.11 Post Observation teardown
If the observation executed successfully, you can use the following commands to reset the telescope.

#### 3.11.1 Clear scan configuration 

In [ ]:
sub.end()
telescope_monitor_plot.show()
print("OK")

#### 3.11.2 Release Subarray resources

In [ ]:
sub.release()
telescope_monitor_plot.show()
print("OK")

### 3.12 Reset the Telescope/Subarray (On Failure)
Set booleans to True to reset the system after a failed execution.

In [ ]:
do_reset_subarray = False
do_reset_dish = False

if do_reset_subarray:
    sub.abort()
    time.sleep(3)
    sub.restart()

if do_reset_dish:
    dish_deployment: TangoDishDeployment
    for dish_deployment in dish_deployments:
        dish_deployment.reset_dish()
print("OK")

## 4 Debug Telescope

### 4.1 Display Telescope State

In [ ]:
telescope_state.state
print("OK")

### 4.2 Display Telescope State Subscriptions

In [ ]:
telescope_state.state_monitor.subscriptions
print("OK")

### 4.3 Display Dish LMC State

In [ ]:
dishes_to_debug = [d.dish_id for d in dish_deployments]
for dish in dish_deployments:
    if dish.dish_id in dishes_to_debug:
        dish.print_diagnostics()
print("OK")

### 4.4 Display CSP LMC State

In [ ]:
sut.print_csp_diagnostics()
print("OK")

### 4.5 Display TMC State

In [ ]:
sut.print_tmc_diagnostics()
print("OK")

### 4.6 Display SDP State

In [ ]:
sut.print_sdp_diagnostics()
print("OK")

## 5 Dashboard Links

### 5.1 Taranta

In [ ]:
print(f"SUT: {sut.taranta_endpoint}")
print(f"Test Equipment: {test_equipment.taranta_endpoint}")
for dish in dish_deployments:
    print(f"Dish LMC {dish.dish_id}: {dish.taranta_endpoint}")
print("OK")

### 5.2 Data Product Dashboard

https://k8s.miditf.internal.skao.int/ska-dpd/dashboard/

### 5.3 SDP Signal Displays

In [ ]:
print(sut.signal_displays_endpoint)
print("OK")

## 6 System Health Displays Tests

### Copied from Jama 2024-03-28

### 6.1 Display the system status and bring the telescope into a known operating state of Standby (SPFC, Dish Structure Controller Simulator, SPFRx, CBF, SDP)

Running steps 0.1 and 2.1-2.6 should get the telescope into a standby state which can be checked by running 3.1 and 3.3.1

### 6.2 Use TMC and LMC dashboards to show status and sensor values from each subsystem. Temperature sensors should be displayed where available.
Run dashboards with links in step 4.2

### 6.3 Use the EDA configuration supplied by each product to configure the EDA in the ITF to subscribe to a range of relevant Tango attribute sensors from each product. Temperature sensors should be displayed where available.

### 6.4 Show that archived sensor values can be displayed or plotted over a configurable time period

### 6.5 Show that several historical sensor values can be plotted on the same axis.

### 6.6 Configure the alarm handler in the ITF to subscribe to a range of relevant Tango attribute sensors from each product.

### 6.7 Use alarm handler to display which sensors are currently in Warning and Alarm status.

### 6.8 Show the drill-down display from the top level system to the products in the system, showing sensor values and associated health and alarm displays

### 6.9 Force an alarm condition and verify that the alarm is reported with the correct timestamp by the alarm handler.  Repeat for various sensors. (it may be necessary to change the alarm threshold values to create the alarm condition)

### 6.10 Perform network interruption conditions (eg. remove a network cable for a period) to demonstrate communication loss alarm indications and recovery.

### 6.11 Display simulated pointing position of antennas and target information